## Data Loading and diagnosis added

In [9]:
import pandas as pd
from src import data_query

In [34]:
diagnosis_df = data_query.load_query(TABLE_NAME='[Episode-Diagnose]', source='BI')

In [35]:
diagnosis_df['VISIT_NO'] = diagnosis_df['Episode_Key'].apply(lambda x:x.split('_')[1])
# diagnosis_df['Hospital_ID'] = diagnosis_df['Episode_Key'].apply(lambda x:x.split('_')[0])

In [36]:
diagnosis_df['VISIT_NO'].dtype

dtype('O')

In [37]:
diagnosis_df

,Episode_Key,ICD10,Diagnosis_Key,VISIT_NO
0,1_1328896-4,J06.9,1_10164354,1328896-4
1,1_1031641-3,J30.4,1_10164418,1031641-3
2,1_65079-107,A09,1_111220949,65079-107
3,1_1326483-4,R22.1,1_10171516,1326483-4
4,1_1325276-8,K02,1_10164622,1325276-8
...,...,...,...,...
38362,1_831810-92,R52.0,1_10171630,831810-92
38363,1_1356893-1,K90.4,1_10165130,1356893-1
38364,1_1363505-9,M54.1,1_111230672,1363505-9
38365,1_1356840-2,M22.2,1_10166537,1356840-2


In [38]:
diagnosis_df.to_excel('data/HJH/Episode_Diagnose.xlsx', index=False)

In [39]:
# read the original parquet file for service and visit
df_claim = pd.read_parquet('data/prq/df.parquet')

In [40]:
df_claim['VISIT_NO'].dtype

dtype('O')

In [42]:
df_claim=df_claim.merge(diagnosis_df,how='left',on='VISIT_NO')

In [45]:
df_claim.ICD10

0           Z36
1         R22.1
2           R53
3           R10
4         R00.1
          ...  
488828    I24.9
488829    R07.4
488830    R07.4
488831    R07.4
488832    I46.0
Name: ICD10, Length: 488833, dtype: object

In [46]:
df_claim.to_parquet('data/prq/df.parquet',index=False)

## Update data

In [1]:
import pandas as pd
from src.data_query import update_data
from src.data_local import read_last_date , create_folder,  store_dfs, load_stored_dfs, merge_claim

In [2]:
FIRST_DATE = read_last_date()
FIRST_DATE

'2024-04-30'

In [3]:
LAST_DATE = '2024-06-12' ## 60 days margin
LAST_DATE

'2024-06-12'

In [4]:
df_visit, df_service,df_diagnose, df_episode = update_data(FIRST_DATE= FIRST_DATE, LAST_DATE=LAST_DATE)

In [5]:
path_date = 'data/HJH/12-06-2024'
create_folder(folder_name= path_date)
store_dfs(df_visit, df_service,df_diagnose,df_episode,path_date)

In [4]:
## required after the loading session
path_date = 'data/HJH/12-06-2024'
df_visit, df_service,df_diagnose, df_episode = load_stored_dfs(path_date)

In [5]:
df_claim = merge_claim(df_visit, df_service, df_episode)

In [8]:
df_claim.to_parquet('data/HJH/12-06-2024/df_claim.parquet')

In [6]:
df_old_claim = pd.read_parquet('data/HJH/30-04-2024/df.parquet')

In [13]:
dtype = type(df_old_claim['VISIT_ID'].iloc[0])
df_claim['VISIT_ID'].astype(dtype)

0         237963
1         237963
2         237963
3         237963
4         237963
           ...  
189287    252519
189288    252519
189289    252519
189290    252519
189291    252519
Name: VISIT_ID, Length: 189292, dtype: int64

In [16]:
for col in df_old_claim.columns:
    try:
        dtype = type(df_old_claim[col].iloc[0])
        df_claim[col] = df_claim[col].astype(dtype)
    except:
        if 'timestamp' in str(dtype):
            df_claim[col] = pd.to_datetime(df_claim[col])

In [17]:
cols = ['VISIT_ID','NET_WITH_VAT', 'SERVICE_DESCRIPTION', 'QTY', 'LINE_CLAIMED_AMOUNT_SAR', 'LINE_ITEM_DISCOUNT', 'NET_VAT_AMOUNT', 'PATIENT_VAT_AMOUNT', 'VAT_PERCENTAGE', 'TREATMENT_TYPE_INDICATOR', 'SERVICE_TYPE', 'DURATION', 'QTY_STOCKED_UOM', 'OASIS_IOS_DESCRIPTION', 'UNIT_PRICE_STOCKED_UOM', 'UNIT_PRICE_NET', 'DISCOUNT_PERCENTAGE', 'EMERGENCY_INDICATOR', 'PROVIDER_DEPARTMENT_CODE', 'PROVIDER_DEPARTMENT', 'DOCTOR_SPECIALTY_CODE', 'DOCTOR_CODE', 'PATIENT_AGE', 'UNIT_OF_AGE', 'PATIENT_NATIONALITY', 'PATIENT_MARITAL_STATUS', 'PATIENT_GENDER', 'CLAIM_TYPE', 'TOTAL_CLAIMED_AMOUNT_SAR', 'TOTAL_DISCOUNT', 'TOTAL_DEDUCTIBLE', 'TOTAL_PATIENT_VATAMOUNT', 'DEPARTMENT_TYPE', 'TREATMENT_TYPE', 'PURCHASER_CODE', 'NEW_BORN', 'ICD10']

df = pd.concat([df_old_claim,df_claim])
df = df.sort_values( by =['CREATION_DATE'])

In [18]:
print(df.shape)
df.drop_duplicates(cols,keep='last',inplace=True)

(678125, 96)


In [19]:
df.shape

(476190, 96)

In [20]:
df

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,ATTACHMENT_TYPE,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,Episode_Key,ICD10,Diagnosis_Key
397551,10335,2024-01-01 09:58:15,194131,575.00,Routine preoperative anaes assessment,1.0,500.00,500.00,0.0,500.00,...,FILE,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
397550,10335,2024-01-01 09:58:15,194131,15346.75,Elective classical caesarean section,1.0,15700.00,15700.00,0.0,15700.00,...,FILE,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
397549,10335,2024-01-01 09:58:15,194131,11.38,TRANDATE INJ. 100 MG.-AMP.20ML,1.0,11.98,11.98,0.0,11.98,...,FILE,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
397543,10335,2024-01-01 09:58:15,194131,312.80,Cross Matching,1.0,340.00,340.00,0.0,340.00,...,FILE,N,H,2023-12-07 10:29:10,2023-12-09 09:52:10,2023-12-01,2023-12-10,1_1335736-8,O82,1_1111231440
397556,10335,2024-01-01 10:00:43,194132,299.00,TSH,1.0,325.00,325.00,0.0,325.00,...,FILE,N,H,2023-12-07 14:09:32,2023-12-09 09:52:28,2023-12-01,2023-12-10,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184614,9638,2024-05-31 18:36:40,252592,95.13,k +,1.0,127.26,127.26,0.0,127.26,...,FILE,N,H,2024-04-16 07:45:09,2024-05-05 22:07:21,2024-05-01,2024-05-15,nan,nan,nan
184615,9638,2024-05-31 18:36:40,252592,95.13,Blood sodium Level,1.0,127.26,127.26,0.0,127.26,...,FILE,N,H,2024-04-16 07:45:09,2024-05-05 22:07:21,2024-05-01,2024-05-15,nan,nan,nan
184616,9638,2024-05-31 18:36:40,252592,0.78,CORDARONE 200MG TAB,1.0,0.84,0.84,0.0,0.84,...,FILE,N,H,2024-04-16 07:45:09,2024-05-05 22:07:21,2024-05-01,2024-05-15,nan,nan,nan
184618,9638,2024-05-31 18:36:40,252592,214.53,IV admin of pharmac agt oth unsp agent,10.0,287.00,287.00,0.0,287.00,...,FILE,N,H,2024-04-16 07:45:09,2024-05-05 22:07:21,2024-05-01,2024-05-15,nan,nan,nan


In [21]:
df.to_parquet('data/HJH/12-06-2024/df.parquet')

In [22]:
df.to_csv('data/HJH/12-06-2024/df.csv',index=False)